In [1]:
import torch, nnunetv2
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("nnU-Net v2 imported successfully!")

# Test that you can access nnunetv2 modules
print("nnunetv2 location:", nnunetv2.__file__)

Torch: 2.5.1+cu121
CUDA available: True
nnU-Net v2 imported successfully!
nnunetv2 location: C:\Users\User\anaconda3\envs\nnunet_new\Lib\site-packages\nnunetv2\__init__.py


In [2]:
import os, nnunetv2, torch

# Work directories - updated for your local setup
BASE = "D:/nnunet_with_classification/data"
RAW  = f"{BASE}/nnUNet_raw"
PREP = f"{BASE}/nnUNet_preprocessed"
RES  = f"{BASE}/nnUNet_results"

# Create nnU-Net directories
os.makedirs(RAW, exist_ok=True)
os.makedirs(PREP, exist_ok=True)
os.makedirs(RES, exist_ok=True)

# Set environment variables
os.environ["nnUNet_raw"] = RAW
os.environ["nnUNet_preprocessed"] = PREP
os.environ["nnUNet_results"] = RES

print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda)
print("nnU-Net v2: imported successfully!")
print(f"nnU-Net directories created at: {BASE}")

Torch: 2.5.1+cu121 | CUDA: 12.1
nnU-Net v2: imported successfully!
nnU-Net directories created at: D:/nnunet_with_classification/data


In [5]:
import os, glob, pathlib, shutil, re, json, csv

# >>>>>>>>>>>> UPDATE THIS PATH <<<<<<<<<<<<
SRC = "D:/nnunet_with_classification/data"

DSID = 777
DSNAME = f"Dataset{DSID:03d}_M31Quiz"
RAW  = os.environ["nnUNet_raw"]
PREP = os.environ["nnUNet_preprocessed"]
DSROOT = f"{RAW}/{DSNAME}"
imgTr = f"{DSROOT}/imagesTr"; lblTr = f"{DSROOT}/labelsTr"; imgTs = f"{DSROOT}/imagesTs"
for d in (imgTr, lblTr, imgTs): os.makedirs(d, exist_ok=True)

def stem(p): return pathlib.Path(p).name.replace(".nii.gz","")
sub_regex = re.compile(r"subtype\s*([012])", re.IGNORECASE)

def find_split(name):
    # case-insensitive split folder lookup
    for cand in os.listdir(SRC):
        if cand.lower() == name:
            p = os.path.join(SRC, cand)
            if os.path.isdir(p): return p
    return None

train_dir = find_split("train")
val_dir   = find_split("validation")
test_dir  = find_split("test")
assert train_dir and val_dir and test_dir, f"Could not find train/validation/test under {SRC}"

def ingest_split(split_dir, cls_map):
    imgs = glob.glob(os.path.join(split_dir, "**", "*_0000.nii.gz"), recursive=True)
    used = 0
    for img in sorted(imgs):
        case = stem(img).replace("_0000","")
        msk  = img.replace("_0000.nii.gz",".nii.gz")
        if not os.path.exists(msk):
            continue
        shutil.copy(img, f"{imgTr}/{case}_0000.nii.gz")
        shutil.copy(msk, f"{lblTr}/{case}.nii.gz")
        used += 1
        # infer subtype from folder names (expects 'subtype0/1/2')
        sub_idx = None
        for part in pathlib.Path(img).parts:
            m = sub_regex.search(part)
            if m:
                sub_idx = int(m.group(1)); break
        if sub_idx is not None:
            cls_map[case] = sub_idx
    return used

cls_map = {}
n_tr = ingest_split(train_dir, cls_map)
n_va = ingest_split(val_dir,   cls_map)

# test images
for img in sorted(glob.glob(os.path.join(test_dir, "**", "*_0000.nii.gz"), recursive=True)):
    case = stem(img).replace("_0000","")
    shutil.copy(img, f"{imgTs}/{case}_0000.nii.gz")

print("imagesTr:", len(glob.glob(f"{imgTr}/*_0000.nii.gz")))
print("labelsTr:", len(glob.glob(f"{lblTr}/*.nii.gz")))
print("imagesTs:", len(glob.glob(f"{imgTs}/*_0000.nii.gz")))
print("Mapped classification labels:", len(cls_map))

# dataset.json
dataset_json = {
  "name": DSNAME,
  "tensorImageSize": "3D",
  "modality": {"0": "CT"},
  "labels": {"background": 0, "pancreas": 1, "lesion": 2},
  "numTraining": len(glob.glob(f"{lblTr}/*.nii.gz")),
  "numTest": len(glob.glob(f"{imgTs}/*_0000.nii.gz")),
  "training": [{"image": f"./imagesTr/{stem(i)}.nii.gz",
                "label": f"./labelsTr/{stem(i).replace('_0000','')}.nii.gz"}
               for i in sorted(glob.glob(f"{imgTr}/*_0000.nii.gz"))],
  "test": [f"./imagesTs/{stem(i)}.nii.gz" for i in sorted(glob.glob(f"{imgTs}/*_0000.nii.gz"))]
}
os.makedirs(DSROOT, exist_ok=True)
with open(f"{DSROOT}/dataset.json","w") as f: json.dump(dataset_json, f, indent=2)

# splits_final.json (use original validation as nnU-Net val)
val_cases = {stem(p).replace("_0000","") for p in glob.glob(os.path.join(val_dir, "**", "*_0000.nii.gz"), recursive=True)}
all_cases = sorted([stem(p).replace("_0000","") for p in glob.glob(f"{imgTr}/*_0000.nii.gz")])
train_cases = [c for c in all_cases if c not in val_cases]
spdir = f"{PREP}/{DSNAME}"
os.makedirs(spdir, exist_ok=True)
with open(f"{spdir}/splits_final.json","w") as f:
    json.dump([{"train": train_cases, "val": sorted(list(val_cases))}], f, indent=2)

# classification_labels.csv
csv_path = f"{spdir}/classification_labels.csv"
with open(csv_path, "w", newline="") as f:
    w = csv.writer(f)
    for c in all_cases:
        if c in cls_map:
            w.writerow([c, cls_map[c]])
        else:
            # if you see many misses here, your folders may not be named 'subtype0/1/2'
            pass

print("Wrote dataset.json, splits_final.json, classification_labels.csv")

imagesTr: 288
labelsTr: 288
imagesTs: 72
Mapped classification labels: 288
Wrote dataset.json, splits_final.json, classification_labels.csv


In [7]:
import json, glob, os, pathlib

DSID = 777
DSNAME = f"Dataset{DSID:03d}_M31Quiz"
RAW   = os.environ["nnUNet_raw"]
DSROOT = f"{RAW}/{DSNAME}"
lblTr = f"{DSROOT}/labelsTr"

dataset_v2 = {
    "channel_names": { "0": "CT" },               # required in v2
    "labels": { "background": 0, "pancreas": 1, "lesion": 2 },  # your classes
    "numTraining": len(glob.glob(os.path.join(lblTr, "*.nii.gz"))),
    "file_ending": ".nii.gz"
}
with open(f"{DSROOT}/dataset.json", "w") as f:
    json.dump(dataset_v2, f, indent=2)

print("✅ Wrote v2 dataset.json at:", f"{DSROOT}/dataset.json")
print("numTraining:", dataset_v2["numTraining"])

✅ Wrote v2 dataset.json at: D:/nnunet_with_classification/data/nnUNet_raw/Dataset777_M31Quiz/dataset.json
numTraining: 288


In [9]:
import nibabel as nib, numpy as np, glob, os, pathlib

DSID = 777
DSNAME = f"Dataset{DSID:03d}_M31Quiz"
lblTr = os.path.join(os.environ["nnUNet_raw"], DSNAME, "labelsTr")

bad = []
for p in sorted(glob.glob(os.path.join(lblTr, "*.nii.gz"))):
    img = nib.load(p)
    arr = img.get_fdata()  # floats possible
    uniq = np.unique(arr)
    if not np.all(np.isin(uniq, [0,1,2])):
        bad.append((p, uniq))

print("Masks needing fix:", len(bad))
for p, uniq in bad[:10]:
    print("  ", pathlib.Path(p).name, "unique:", uniq[:10])

# Fix: round and clip to {0,1,2}, write as uint8
for p, uniq in bad:
    img = nib.load(p)
    arr = img.get_fdata()
    arr = np.rint(arr)           # round to nearest integer
    arr = np.clip(arr, 0, 2)     # enforce label set
    arr = arr.astype(np.uint8)

    hdr = img.header.copy()
    hdr.set_data_dtype(np.uint8)
    # avoid unintended scaling
    hdr["scl_slope"] = 1
    hdr["scl_inter"] = 0

    fixed = nib.Nifti1Image(arr, img.affine, hdr)
    nib.save(fixed, p)

# Sanity check after fix
remaining = []
for p in sorted(glob.glob(os.path.join(lblTr, "*.nii.gz"))):
    arr = nib.load(p).get_fdata()
    uniq = np.unique(arr)
    if not np.all(np.isin(uniq, [0,1,2])):
        remaining.append((p, uniq))
print("Remaining masks with non-{0,1,2} labels:", len(remaining))

Masks needing fix: 214
   quiz_0_060.nii.gz unique: [0.         1.00001526 2.        ]
   quiz_0_066.nii.gz unique: [0.         1.00001526 2.        ]
   quiz_0_077.nii.gz unique: [0.         1.00001526 2.        ]
   quiz_0_117.nii.gz unique: [0.         1.00001526 2.        ]
   quiz_0_126.nii.gz unique: [0.         1.00001526 2.        ]
   quiz_0_139.nii.gz unique: [0.         1.00001526 2.        ]
   quiz_0_145.nii.gz unique: [0.         1.00001526 2.        ]
   quiz_0_150.nii.gz unique: [0.         1.00001526 2.        ]
   quiz_0_159.nii.gz unique: [0.         1.00001526 2.        ]
   quiz_0_160.nii.gz unique: [0.         1.00001526 2.        ]
Remaining masks with non-{0,1,2} labels: 0


In [ ]:
!nnUNetv2_plan_and_preprocess -d 777 -c 3d_fullres --verify_dataset_integrity -pl nnUNetPlannerResEncM

In [ ]:
!nnUNetv2_train 777 3d_fullres 0 -tr NNUNet -p nnUNetResEncUNetMPlans